In [3]:
import json
import pandas as pd
from sentence_transformers.util import cos_sim  
from sentence_transformers import SentenceTransformer as SBert
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
from numpy import sqrt
import numpy as np
import os
from tqdm import trange
# modelName = 'princeton-nlp/sup-simcse-bert-base-uncased'
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

dataName1 = "Mafengwo"
dataName2 = "Tripadvisor"

with open("../user_data/%s_addr.json"%dataName1, "r", encoding="utf-8") as f:
    data1 = json.load(f)
with open("../user_data/%s_addr.json"%dataName2, "r", encoding="utf-8") as f:
    data2 = json.load(f)
print(len(data1), len(data2))

259 591


In [2]:
# with open("../user_data/香港十八区.json", "r", encoding="utf-8") as f:
#     districtSet = json.load(f)

# stat1 = [0 for _ in range(19)]
# stat2 = [0 for _ in range(19)]
# for d in data1:
#     dist = d["行政区"]
#     if dist in districtSet:
#         stat1[districtSet.index(dist)] += 1
#     else:
#         stat1[-1] += 1
# for d in data2:
#     dist = d["行政区"]
#     if dist in districtSet:
#         stat2[districtSet.index(dist)] += 1
#     else:
#         stat2[-1] += 1
# print(stat1)
# print(stat2)

[53, 17, 6, 18, 30, 1, 10, 4, 1, 5, 8, 7, 16, 28, 5, 5, 0, 25, 20]
[109, 29, 22, 25, 56, 20, 9, 14, 10, 14, 12, 15, 15, 21, 21, 16, 14, 17, 152]


In [4]:
with open("../user_data/districtList.json", "r", encoding="utf-8") as f:
    districtSet = json.load(f)

def combineEnt(d1, d2, district):
    rnt = {"id": "%s-%s"%(d1["id"], d2["id"])}
    rnt["行政区"] = district
    for attr in list(d1.keys())[2:]:
        rnt[attr] = [d1[attr], d2[attr]]
    return rnt

result = []
for d1 in data1:
    if d1["行政区"] not in districtSet and d1["行政区"] != "U":
        continue
    for d2 in data2:
        if d2["行政区"] not in districtSet and d2["行政区"] != "U":
            continue
        # 任意一个实体的行政区未知，则进行匹配
        if d1["行政区"] == "U" or d2["行政区"] == "U":
            result.append(combineEnt(d1, d2, "U"))
        # 两个实体的行政区一致，也进行匹配
        elif d1["行政区"] == d2["行政区"]:
            result.append(combineEnt(d1, d2, d1["行政区"]))
print(len(result))

48428


In [5]:
# Do word vector calculations on all occurrences of strings in advance
# to reduce the time complexity of subsequent processes
if os.path.exists("pre_dict.json"):
    print("Preprocessing dictionary file already exists!")
else:
    eDict = {}
    cols = ["Name", "Address", "District", "RoadInfo", "Poi", "RoomInfo"]
    data = data1 + data2
    for i in trange(len(data)):
        d = data[i]
        for col in cols:
            target = d[col]
            if target == None or target == "": continue
            if target not in eDict:
                eDict[target] = model.encode(target).tolist()
    with open("pre_dict.json", "w", encoding="utf-8") as f:
        json.dump(eDict, f, ensure_ascii=False, indent=2)

100%|██████████| 850/850 [00:26<00:00, 32.51it/s]


In [6]:
from geopy.distance import geodesic

with open("pre_dict.json", "r", encoding="utf-8") as f:
    eDict = json.load(f)

def calcDist(pos1, pos2):
    if pos1[0] is None or pos2[0] is None:
        return None
    return geodesic((pos1[1], pos1[0]), (pos2[1], pos2[0])).kilometers

def calcSimi(lst1, lst2):
    lstCosine = []
    for i in range(len(lst1)):
        if lst1[i] == "" or lst2[i] == "":
            lstCosine.append(None)
        else:
            e1 = eDict[lst1[i]]
            e2 = eDict[lst2[i]]
            sim = 1 - cosine(e1, e2)
            lstCosine.append(sim)
    return lstCosine

cols = ["id", "行政区", "Location", "Name", "Address", "District", "RoadInfo", "Poi", "RoomInfo"]
dfList = []
cnt = 0
for i in trange(len(result)):
    d = result[i]
    lst = [d["id"], d["行政区"]]
    lst.append(calcDist(d["Location"][0], d["Location"][1]))
    lst1 = [d[attr][0] for attr in cols[3:]]
    lst2 = [d[attr][1] for attr in cols[3:]]
    lst = lst + calcSimi(lst1, lst2)
    dfList.append(lst)
df = pd.DataFrame(dfList, columns=cols)
df = df.fillna(value=np.nan)
df.to_csv("../user_data/data.csv", encoding="utf-8")

100%|██████████| 48428/48428 [00:10<00:00, 4518.06it/s]
